In [1]:
#Mango_testing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [3]:
# Define data generators for train, validation and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/mango_test/train2',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/mango_test/train2',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/mango_test/pred-20240215T041610Z-001',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 136 images belonging to 2 classes.
Found 33 images belonging to 2 classes.
Found 14 images belonging to 1 classes.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [5]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [6]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
5/5 [==============================] - 36s 6s/step - loss: 1.7968 - accuracy: 0.3971 - val_loss: 0.6827 - val_accuracy: 0.6970
Epoch 2/5
5/5 [==============================] - 17s 4s/step - loss: 0.6978 - accuracy: 0.6471 - val_loss: 0.7088 - val_accuracy: 0.4545
Epoch 3/5
5/5 [==============================] - 18s 4s/step - loss: 0.6157 - accuracy: 0.5441 - val_loss: 0.7195 - val_accuracy: 0.4848
Epoch 4/5
5/5 [==============================] - 17s 3s/step - loss: 0.4184 - accuracy: 0.9412 - val_loss: 1.0030 - val_accuracy: 0.5455
Epoch 5/5
5/5 [==============================] - 19s 4s/step - loss: 0.2595 - accuracy: 0.9191 - val_loss: 1.1445 - val_accuracy: 0.4545


In [7]:
model.save("Model.h5","label.txt")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
#test your image
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

#load the model
model = load_model('/content/Model.h5')
#classes
class_names = ['This mango is rippen mango','This mango is unrippen mango']
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)



In [14]:
image = Image.open("/content/drive/MyDrive/mango_test/pred-20240215T041610Z-001/Neelam/edit5.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array


In [13]:
# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
confidence_score = prediction[0][index]
# Print prediction and confidence score
print("Result: ", class_names[index], end="")
print("\n")
print("Accuracy: ", confidence_score)


1/1 [==============================] - 0s 110ms/step
Result:  This mango is rippen mango

Accuracy:  0.5571335
